In [ ]:
import sys
sys.path.append("..")
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np

# this is the convenience function
from autokoopman import auto_koopman

In [ ]:
# for a complete example, let's create an example dataset using an included benchmark system
import autokoopman.benchmark.fhn as fhn
fhn = fhn.FitzHughNagumo()
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(10, 2)),
    tspan=[0.0, 10.0],
    sampling_period=0.1
)

In [ ]:
from autokoopman.observable import RFFObservable, PolynomialObservable, IdentityObservable

# augment (combine) observables function
# in this case, combine multiple lengthscales together
my_obs = IdentityObservable() | RFFObservable(2, 50, 0.1) | RFFObservable(2, 50, 1.0)
my_obs(np.random.rand(10, 2)).shape

In [ ]:
# learn model from data
experiment_results = auto_koopman(
    training_data,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type=my_obs,         # use our custom observables
    opt="grid",             # grid search to find best hyperparameters
    n_obs=200,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# extract the tuned model
model = experiment_results['tuned_model']

In [ ]:
# get evolution matrices
A, B = model.A, model.B

In [ ]:
# states -> obs func
model.obs_func(np.random.rand(10, 2)).shape

In [ ]:
# getting the observables parameters is uglier because it depends what they are
# for the first RFF
model.obs_func.observables[0].observables[1].u
model.obs_func.observables[0].observables[1].w

# for the second
model.obs_func.observables[1].u
model.obs_func.observables[1].w

In [ ]:
# get the model from the experiment results
model = experiment_results['tuned_model']

# simulate using the learned model
iv = [0.5, 0.1]
trajectory = model.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

In [ ]:
# simulate the ground truth for comparison
true_trajectory = fhn.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

plt.figure(figsize=(10, 6))

# plot the results
plt.plot(*trajectory.states.T, label='Trajectory Prediction')
plt.plot(*true_trajectory.states.T, label='Ground Truth')

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.grid()
plt.legend()
plt.title("FHN Test Trajectory Plot")
plt.show()